In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Liver_Cancer/GSE82302'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Blood exsosomal miRNAs of liver disease patients"
!Series_summary	"Aiming to develop circulatin biomarkers for liver disease, including  progression of fibrosis/inflammation."
!Series_overall_design	"40 liver disease patients"
Sample Characteristics Dictionary:
{0: ['subject status/id: ICC patient 1', 'subject status/id: ICC patient 2', 'subject status/id: ICC patient 3', 'subject status/id: ICC patient 4', 'subject status/id: ICC patient 5', 'subject status/id: ICC patient 6', 'subject status/id: ICC patient 7', 'subject status/id: ICC patient 8', 'subject status/id: ICC patient 9', 'subject status/id: ICC patient 10', 'subject status/id: ICC patient 11', 'subject status/id: ICC patient 12', 'subject status/id: ICC patient 13', 'subject status/id: ICC patient 14', 'subject status/id: CHC patient 1', 'subject status/id: CHC patient 2', 'subject status/id: CHC patient 3', 'subject status/id: CHC patient 4', 'subject status/id: HCC patient 1', 'subj

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_age = None  # define the functions when applicable

# Determine if gene expression data is available
is_gene_available = False  # Based on the series title and design, it seems to be miRNA data.

# Determine availability of the variables

# Liver_Cancer (trait) data 
# No explicit field for liver cancer type; inferred from subject status. Different liver diseases mentioned but no single value indicating Liver_Cancer. 
trait_row = None

# Age data
if 1 in {1: ['age: 80', 'age: 66', 'age: 39', 'age: 31', 'age: 68', 'age: 34', 'age: 63', 'age: 50', 'age: 62', 'age: 77', 'age: 73', 'age: 78', 'age: 45', 'age: 51', 'age: 71', 'age: 72', 'age: 75', 'age: 88', 'age: 67', 'age: 84', 'age: 83', 'age: 85', 'age: 64', 'age: 79', 'age: 69']}:
    age_row = 1
else:
    age_row = None

# Gender data
if 2 in {2: ['gender: male', 'gender: female']}:
    gender_row = 2
else:
    gender_row = None

# Define conversion functions

def convert_trait(value):
    return None  # Not applicable

def convert_age(value):
    try:
        return float(value.split(': ')[1])
    except (ValueError, IndexError):
        return None

def convert_gender(value):
    gender_str = value.split(': ')[1].strip().lower()
    if gender_str == 'male':
        return 1
    elif gender_str == 'female':
        return 0
    else:
        return None

# Save cohort information
save_cohort_info('GSE82302', './preprocessed/Liver_Cancer/cohort_info.json', is_gene_available, trait_row is not None)

# Clinical Feature Extraction
if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Liver_Cancer', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Liver_Cancer/trait_data/GSE82302.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))


A new JSON file was created at: ./preprocessed/Liver_Cancer/cohort_info.json
